# Pràctica 5 PLN: classificació de text (multiclasse).
En aquesta pràctica entrenarem un classificador de text per a diversos problemes multiclasse.\
Provarem amb una extracció de característiques vectorials *sparse* i amb una mitjana de *word vectors*.

### Noms:
Alejandro Madrid Galarza \
Antonio José López Martínez

## Part 0: Actualitzar paqueteria `conda`i instal·lar model llenguatge.
Executar les següents sentències a la terminal per a instal·lar el model de llenguatge que usarem en aquesta pràctica.

> `python -m spacy download en_core_web_md`
> `python -m spacy download es_core_news_sm`

## Part 1: Conjunt de textos de "mundocine".
En aquesta part el conjunt de dades que utilitzarem consisteix en una sèrie de crítiques de pel·lícules de cinema, emmagatzemades en format XML (una crítica per arxiu). Hem preparat una funció de tipus `generator` que processa el directori on estan els arxius de les crítiques i torna per cada arxiu XML una tupla amb 4 valors:
 - Nom de la pel·lícula (*string*)
 - Resum breu de la crítica (*string*)
 - Text de la crítica (*string*)
 - Valoració de la pel·lícula (*int* de 1 a 5)

In [77]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """Generador que llegeix el contingut dels fitxers XML d'una carpeta.
    Retorna el <body> de la <review> a cada fitxer XML.
    Fitxers XML codificats com a 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(re.sub(r'(<>)|&|(<-)', '', f.read()))

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


### Extracció de característiques.
Tenim dos directoris, un directori de train (`"critiques/train"`) i un altre directori de test (`"critiques/test"`).\
Abans de processar el text calcularem una sèrie de paràmetres de cada crítica. \
Per això processem cada crítica per i guardarem els resultats en un objecte `DataFrame` de Pandes.\
Com a característica de cada crítica extreurem:
- Títol de la pel·lícula.
- Longitud (en caràcters) del resum.
- Longitud (en caràcters) del text de la crítica.
- Puntuació de la crítica.

### Exercici
Completa el codi següent per generar el `DataFrame` sobre el conjunt de train*.

In [78]:
import pandas as pd

# Creem una llista en blanc.
dades = []

# Recorrem les crítiques i calculem les seves mètriques.
for c in parse_folder('critiques/critiques/train'):
    dades.append({
        'títol': c[0],
        'LongResum': len(c[1]),
        'LongCritica': len(c[2]),
        'puntuació': c[3]
    })

resum = pd.DataFrame(dades)

In [79]:
resum.describe()

,LongResum,LongCritica,puntuació
count,2714.000000,2714.000000,2714.000000
mean,173.058585,2805.424834,3.060796
std,61.891358,1675.839317,1.141479
min,12.000000,0.000000,1.000000
25%,129.000000,1751.000000,2.000000
50%,173.000000,2358.500000,3.000000
75%,217.000000,3427.500000,4.000000
max,425.000000,26668.000000,5.000000


### Neteja de text.
Prepararem aquest conjunt de textos per entrenar un model per predir la puntuació de cada crítica a partir del text de la crítica.\
Realitzarem el següent processat:
- Introduïm un espai després de determinats signes de puntuació (".", "?") perquè el tokenitzat siga correcte.
- Separar el text a *tokens*.
- Eliminar els *tokens* de tipus *stop-word*, signes de puntuació, espais o amb longitud de 1.
- Convertir les entitats de tipus `PER` al token *persona*.
- Lematitzar el text i convertir minúscules.

In [80]:
import spacy

nlp = spacy.load("es_core_news_sm")
nlp.add_pipe("merge_entities")

def normalitzar(text):
    # Separem després de certs signes de puntuació.
    text = re.sub(r"([\.\?])", r"\1 ", text)
    doc = nlp(text)
    tokens = [t.lemma_.lower() if not t.ent_type_=='PER' else '_PERSONA_'
              for t in doc if not t.is_punct and not t.is_stop and not t.is_space and len(t.text)>1]
    output = ' '.join(tokens)
    
    return output

Comprova el seu funcionament a la primera crítica del conjunt de train*.

In [81]:
ruta_fitxer = "critiques/critiques/train/2.xml"

# Llegim el contingut de la crítica
with open(ruta_fitxer, 'r', encoding='latin1') as file:
    primera_critica = file.read()

# Aplicar normalització
critica_neta = normalitzar(primera_critica)

print("Text original de la primera crítica:")
print(primera_critica)
print("\nText normalitzat de la primera crítica:")
print(critica_neta)

Text original de la primera crítica:
<review author="Torbe" title="La guerra de los mundos" rank="1" maxRank="5" source="muchocine">
	<summary>Hasta los cojones de los yankis</summary>
	<body>Cada vez me gusta menos el cine de masas. Las peliculas que ven todo el mundo me parecen cada vez mas coñazo y mas insufribles. No se porqué pero siempre el prota es tonto del culo y tiene suerte, y al final de la peli, cuando ha logrado vencer al mal, se convierte en listo, y las chorradas que hacia al comienzo de la pelicula se esfuman como por arte de magia. Se vuelve maduro e inteligente.Esta peli de Spielberg es mas de lo mismo, huir y huir y que no le den ni un solo tiro. Además el cabron ha metido a un par de actores que es como para echarles de comer aparte. La niña, una vieja metida en el cuerpo de una niña, porque solo hay que verle hablar (en version original claro) para darse cuenta que estamos ante uno de los grandes freaks del cine. Se creeran que hace gracia la nena cuando habla igu

### Extracció de característiques *sparse*.
Calcularem les matrius de característiques *bag-of-words* i *tfidf* del conjunt de textos anterior.\
Farem servir la llibreria `scikit-learn` per vectoritzar els documents.

In [82]:
# Per no haver de carregar totes les crítiques en memòria, creem un generador que torna iterativament el
# text processat de cada crítica.

def critiques_folder(folder):
    for c in parse_folder(folder):
        yield normalitzar(c[2])

Comprova'n el funcionament generant el text normalitzat de la primera crítica del conjunt de *train*

In [83]:
folder_train = "critiques/critiques/train"
# 
# Inicialitzem el generador per obtenir el text normalitzat de les crítiques del conjunt de train
generador_critiques = critiques_folder(folder_train)

# Obtenim el text normalitzat de la primera crítica
text_normalitzat_primera_critica = next(generador_critiques)

print("Text normalitzat de la primera crítica del conjunt de train:")
print(text_normalitzat_primera_critica)

Text normalitzat de la primera crítica del conjunt de train:
gustar cine masa pelicula ver mundo parecer coñazo insufribl porqué prota tonto culo suerte peli lograr vencer convertir listo chorrada comienzo pelicula esfumar arte magia volver maduro inteligente peli _PERSONA_ huir huir dar tiro cabron meter par actor echarl comer aparte el niña viejo metida cuerpo niña ver él hablar version original dar él freaks cine creeran gracia nena puta madre causar pavor cria persona maduro horroroso el niño niño ver él rol asusta luego este el hijo adolescente cruise subnormal dar él bofetada ver hueso mano _PERSONA_ reflejo denominar manipulacion militar chico matar bicho ningun arma hale loco pensar venir saco quereis decir fanatismo locura alguien queír luchar medio muerte seguro jodar sobremanera pelicula aparezco mongo abuelo familia salir dio viejo aparecer traje domingo maquillado ¿pero que sinsorgado faltar _PERSONA_ volver exmujer mundo tranquilo contar peli pasar peli acabar restar mist

Vectoritzem tot el conjunt de dades usant les funcions de `scikit-learn`.\
Aquestes funcions admeten un objecte `generator` com a argument dʻentrada.\
El vectoritzador ha d'aprendre només sobre el conjunt de TRAIN i després aplicar-se al conjunt de TEST.

In [84]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vect = CountVectorizer(min_df=0.01)

X_train_vectorized = vect.fit_transform(critiques_folder("critiques/critiques/train"))
X_train_vectorized.shape

(2714, 2746)

Aplica el vectoritzador al conjunt de TEST:

In [85]:
X_test_vectorized = vect.transform(critiques_folder("critiques/critiques/test"))
X_test_vectorized.shape

(1164, 2746)

### Variable *target*.
Creem la llista d'etiquetes de cada crítica a partir dels fitxers. Definim 3 classes per a les 5 puntuacions per facilitar-ne la classificació.

In [86]:
puntuacions = {1: 'NEG', 2:'NEG', 3:'NEU', 4:'POS', 5:'POS'}
y_train = []
for c in parse_folder("critiques/critiques/train"):
    y_train.append(puntuacions[c[3]])
    
len(y_train)

2714

#### Exercici.
Crea la llista d'etiquetes de test `y_test` a partir dels fitxers al directori `"critiques/test"`.

In [87]:
puntuacions = {1: 'NEG', 2:'NEG', 3:'NEU', 4:'POS', 5:'POS'}
y_test = []
for c in parse_folder("critiques/critiques/test"):
    y_test.append(puntuacions[c[3]])
    
len(y_test)

1164

### Entrenament del classificador.
Entrenarem amb diferents models de classificador sobre el conjunt de crítiques i compara el seu rendiment.\
#### Exercici
Prova a classificar les crítiques amb els models de regressió logística, Naïve Bayes i SVM lineal.\
Utilitza la mètrica `classification_report` de `scikit-learn` per comparar els models.

In [88]:
#Modelo BoW-LR
# COMPLETAR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Creem un classificador de regressió logística
clf_lr = LogisticRegression()

# Entrenem el classificador amb les dades vectoritzades del conjunt de train
clf_lr.fit(X_train_vectorized, y_train)

# Fem prediccions sobre les dades vectoritzades del conjunt de test
prediccions_lr = clf_lr.predict(X_test_vectorized)

# Avaluem el rendiment del classificador utilitzant classification_report
informe_lr = classification_report(y_test, prediccions_lr)

# Mostrem l'informe de classificació
print("Informe de classificació per a la regressió logística amb Bag-of-Words:\n", informe_lr)


Informe de classificació per a la regressió logística amb Bag-of-Words:
               precision    recall  f1-score   support

         NEG       0.65      0.62      0.63       395
         NEU       0.45      0.47      0.46       374
         POS       0.59      0.60      0.60       395

    accuracy                           0.56      1164
   macro avg       0.57      0.56      0.56      1164
weighted avg       0.57      0.56      0.57      1164



C:\Users\ajlmt\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [89]:
#Modelo BoW-NB
# COMPLETAR
from sklearn.naive_bayes import MultinomialNB

# Creem un classificador de Naïve Bayes
clf_nb = MultinomialNB()

# Entrenem el classificador amb les dades vectoritzades del conjunt de train
clf_nb.fit(X_train_vectorized, y_train)

# Fem prediccions sobre les dades vectoritzades del conjunt de test
prediccions_nb = clf_nb.predict(X_test_vectorized)

# Avaluem el rendiment del classificador utilitzant classification_report
informe_nb = classification_report(y_test, prediccions_nb)

# Mostrem l'informe de classificació
print("Informe de classificació per a Naïve Bayes amb Bag-of-Words:\n", informe_nb)


Informe de classificació per a Naïve Bayes amb Bag-of-Words:
               precision    recall  f1-score   support

         NEG       0.68      0.68      0.68       395
         NEU       0.51      0.45      0.48       374
         POS       0.59      0.66      0.62       395

    accuracy                           0.60      1164
   macro avg       0.59      0.60      0.59      1164
weighted avg       0.60      0.60      0.60      1164



In [90]:
#Modelo BoW-SVM
# COMPLETAR
from sklearn.svm import LinearSVC

# Creem un classificador SVM lineal
clf_svm = LinearSVC()

# Entrenem el classificador amb les dades vectoritzades del conjunt de train
clf_svm.fit(X_train_vectorized, y_train)

# Fem prediccions sobre les dades vectoritzades del conjunt de test
prediccions_svm = clf_svm.predict(X_test_vectorized)

# Avaluem el rendiment del classificador utilitzant classification_report
informe_svm = classification_report(y_test, prediccions_svm)

# Mostrem l'informe de classificació
print("Informe de classificació per a SVM lineal amb Bag-of-Words:\n", informe_svm)


C:\Users\ajlmt\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Informe de classificació per a SVM lineal amb Bag-of-Words:
               precision    recall  f1-score   support

         NEG       0.62      0.58      0.60       395
         NEU       0.45      0.45      0.45       374
         POS       0.56      0.59      0.58       395

    accuracy                           0.54      1164
   macro avg       0.54      0.54      0.54      1164
weighted avg       0.54      0.54      0.54      1164



C:\Users\ajlmt\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Part 2: conjunt "20Newsgroups".
En aquesta part crearem un classificador de textos multiclasse en anglès usant `scikit-learn` usant models BoW, TF-IDF i *averaged word vectors*.

Farem servir com a conjunt de prova el dataset *20newsgroups*, que consisteix en unes 18000 notícies d'un fòrum de discussió en anglès dividides en 20 categories.

### Descàrrega del dataset.
Ens descarreguem el dataset amb la llibreria `scikit-learn`.

In [91]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

def get_data():
    data = fetch_20newsgroups(subset='all',
                              shuffle=True,
                              remove=('headers', 'footers', 'quotes'))
    return data
    
def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []
    for doc, label in zip(corpus, labels):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)

    return filtered_corpus, filtered_labels
    
    
dataset = get_data()

corpus, labels = dataset.data, dataset.target
corpus, labels = remove_empty_docs(corpus, labels)

print(f'\nCarregats {len(corpus)} documents')
print('Classes:\n',dataset.target_names)
print('\nDocument exemple:\n', corpus[10])
print(f'\nClasse: {labels[10]} ({dataset.target_names[labels[10]]})')


Carregats 18331 documents
Classes:
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

Document exemple:
 the blood of the lamb.

This will be a hard task, because most cultures used most animals
for blood sacrifices. It has to be something related to our current
post-modernism state. Hmm, what about used computers?

Cheers,
Kent

Classe: 19 (talk.religion.misc)


La quantitat de documents dins de cada classe està prou balancejada:

In [92]:
from collections import Counter
import pandas as pd

pd.DataFrame([(dataset.target_names[k], v) for k,v in Counter(labels).items()], columns=['clase', 'N'])

,clase,N
0,rec.sport.hockey,975
1,comp.sys.ibm.pc.hardware,964
2,talk.politics.mideast,919
3,comp.sys.mac.hardware,929
4,sci.electronics,958
5,talk.religion.misc,606
6,sci.crypt,962
7,sci.med,960
8,alt.atheism,779
9,rec.motorcycles,969


### Submostreig del conjunt de dades.
Per no treballar amb un conjunt tan gran ens quedarem amb només 2000 mostres.

In [93]:
from sklearn.utils import resample

corpus_sm, labels_sm = resample(corpus, labels, n_samples=2000, replace=False,random_state=123)

#### Exercici.
Divideix el conjunt d'entrenament submostrejat en entrenament i test (30%) per entrenar i validar els models.

In [94]:
# COMPLETAR
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus_sm, labels_sm, test_size=0.3, random_state=42)

print(f'\nTamany de TRAIN: {len(train_corpus)}')
print(f'Tamany de TEST: {len(test_corpus)}')


Tamany de TRAIN: 1400
Tamany de TEST: 600


Mostra quantes mostres de cada classe tenim a TRAIN:

In [95]:
# COMPLETAR
train_df = pd.DataFrame([(dataset.target_names[label], count) for label, count in Counter(train_labels).items()], columns=['Classe', 'Quantitat'])

print("Quantitat de mostres de cada classe a TRAIN:")
print(train_df)

Quantitat de mostres de cada classe a TRAIN:
                      Classe  Quantitat
0            sci.electronics         76
1               misc.forsale         66
2      comp.sys.mac.hardware         68
3         talk.politics.misc         51
4                alt.atheism         50
5    comp.os.ms-windows.misc         88
6   comp.sys.ibm.pc.hardware         74
7             comp.windows.x         77
8                  sci.space         89
9                  rec.autos         77
10        rec.sport.baseball         68
11    soc.religion.christian         65
12     talk.politics.mideast         76
13                   sci.med         68
14           rec.motorcycles         77
15        talk.politics.guns         64
16          rec.sport.hockey         77
17        talk.religion.misc         48
18                 sci.crypt         71
19             comp.graphics         70


### Pre-processament del text.
Realitzem una neteja del text (treiem signes de puntuació i espais) i ens quedem amb el lema de cada paraula en minúscules.

In [96]:
import spacy
import re
import string

nlp=spacy.load('en_core_web_md')

def normalize_document(doc):
    '''Netegem i normalitzem un document passat com a string'''
    # tokenizamos el texto
    tokens = nlp(doc)
    # quitamos puntuación/espacios
    filtered_tokens = [t for t in tokens if not t.is_punct and not t.is_space and not t.is_digit]
    #cogemos el lemma
    lemmas = []
    for tok in filtered_tokens:
        lemma = re.sub('[{}]'.format(re.escape(string.punctuation)), '', tok.lemma_.lower()) if tok.lemma_ != "-PRON-" else tok.lower_
        if len(lemma)>2:
            lemmas.append(lemma)
    # juntamos de nuevo en una cadena
    doc = ' '.join(lemmas)
    return doc

def normalize_corpus(corpus):
    '''Apliquem la funció de normalització sobre el corpus passat com a llista de string'''
    return [normalize_document(text) for text in corpus]

Per exemple veiem el document núm. 15 normalitzat.

In [97]:
# original
print(corpus_sm[15])


Because I'm a guy and most of my pillions are female. 

Also, the other reasons, like having an idea where you passengers
weight is, it being a more comfortable position for the passenger,
and it being a more stable configuration all come into it as well.

Holding the grab rail is a great idea only for braking, when you
don't want the pillion to slide forward into you, otherwise I don't
find it works well.


In [98]:
# normalitzat
print(normalize_document(corpus_sm[15]))

because guy and most pillion female also the other reason like have idea where you passenger weight more comfortable position for the passenger and more stable configuration all come into well hold the grab rail great idea only for braking when you not want the pillion slide forward into you otherwise not find work well


Normalitzem tot el conjunt de textos. Ho guardem com una llista en lloc de `generator` perquè l'hem d'utilitzar múltiples vegades i no volem haver de normalitzar tot el corpus cada cop.

In [99]:
norm_train_corpus = list(map(normalize_document, train_corpus))

#### Exercici.
Crea una llista per al conjunt de TEST normalitzat.

In [100]:
# COMPLETAR
# Normalitzem tot el conjunt de textos de test
norm_test_corpus = list(map(normalize_document, test_corpus))


## Models BoW i TF-IDF.
Instanciem els vectoritzadors per obtenir les característiques BoW i TF-IDF.  
Fem servir el paràmetre max_df=0.9 per eliminar els stop-words com les paraules que apareixen almenys al 90% dels documents i el paràmetre min_df=0.01 per eliminar les paraules que no apareixen almenys en un 1% dels documents.\
Fem servir el model `TfidfTransformer` per calcular la matriu TF-IDF a partir del BoW i no haver de repetir tot l'entrenament.

In [101]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

bow_vectorizer = CountVectorizer(min_df=0.01, max_df=0.9)

tfidf_vectorizer = TfidfTransformer()

## Model Averaged Word Vectors.  
Per calcular els models basats en WV fem servir el model gloVe pre-entrenat a `spaCy`.  

Calculem dos models basats en word-vectors:  
* el vector mitjana dels WV de tots els tokens amb el mateix pes per a totes les paraules.  
* ponderant el WV de cada paraula pel terme de freqüència inversa de document (IDF).  

Definim les funcions per calcular aquestes dues matrius de característiques.

In [115]:
def averaged_word_vectorizer(corpus):
    '''Aplica la funció de càlcul del WV mitjana a tots els
     documents del corpus (corpus és una llista de docs com a strings)'''
    features = [nlp(doc).vector
                    for doc in corpus]
    return np.array(features)

def tfidf_wtd_avg_word_vectors(doc, word_tfidf_map):
    '''Aplica la funció de càlcul del WV ponderat per TF-IDF
     de cada token a un document'''
    tokens = doc.split()

    feature_vector = np.zeros((nlp.vocab.vectors_length,),dtype="float64")
    wts = 0.      
    for word in tokens:
        if nlp.vocab[word].has_vector and word_tfidf_map.get(word, 0): #sólo considera palabras conocidas
            weighted_word_vector = word_tfidf_map[word] * nlp.vocab[word].vector
            wts = wts + 1
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector
    
def tfidf_weighted_averaged_word_vectorizer(corpus, word_tfidf_map):
    '''Aplica la funció de càlcul del WV ponderat per TF-IDF a tots els
     documents del corpus'''                                       
    features = [tfidf_wtd_avg_word_vectors(doc, word_tfidf_map)
                    for doc in corpus]
    return np.array(features)

### Extracció de característiques.
Extraiem característiques amb els diferents models al nostre conjunt d´entrenament.
#### Exercici.
Extreu les característiques BoW, TF-IDF, *averaged Word Vectors* i *TFIDF-weighted WV* dels conjunts normalitzats de train i test.

In [103]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Definir el vectorizador BoW
bow_vectorizer = CountVectorizer(min_df=0.01, max_df=0.9)

# Características Bag of Words
bow_train_features = bow_vectorizer.fit_transform(norm_train_corpus)  
bow_test_features = bow_vectorizer.transform(norm_test_corpus) 

# Definir y aplicar el transformador TF-IDF
tfidf_transformer = TfidfTransformer()
tfidf_train_features = tfidf_transformer.fit_transform(bow_train_features)
tfidf_test_features = tfidf_transformer.transform(bow_test_features)

# Características Averaged Word Vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                   
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)

# Características TF-IDF Weighted Average Word Vector
word_tfidf_map = {key:value for (key, value) in zip(bow_vectorizer.get_feature_names_out(), tfidf_transformer.idf_)}
tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(norm_train_corpus, word_tfidf_map)
tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(norm_test_corpus, word_tfidf_map)

In [104]:
bow_train_features.shape

(1400, 1237)

In [105]:
tfidf_test_features.shape

(600, 1237)

In [106]:
avg_wv_train_features.shape

(1400, 300)

In [107]:
tfidf_wv_test_features.shape

(600, 300)

### Classificació.
Apliquem diferents classificadors a cada model per veure quin funciona millor amb les nostres dades.  
Primer definim unes funcions per entrenar i mesurar el rendiment dels classificadors:

In [108]:
from sklearn import metrics

def get_metrics(true_labels, predicted_labels):
    """Calculem diferents mètriques sobre el
     rendiment del model. Torna un diccionari
     amb els paràmetres mesurats"""
    
    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        3),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3)}
                        

def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    """Funció que entrena un model de classificació sobre
     un conjunt d'entrenament, ho aplica sobre un conjunt
     de test i torna la predicció sobre el conjunt de test
     i les mètriques de rendiment"""
    # genera modelo    
    classifier.fit(train_features, train_labels)
    # predice usando el modelo sobre test
    predictions = classifier.predict(test_features) 
    # evalúa rendimiento de la predicción   
    metricas = get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    return predictions, metricas    

Anem a entrenar sobre el conjunt de train i avaluem al conjunt de test. Desem mètriques en una llista i resultats en una altra per mostrar resum.
#### Exercici.
Completa el codi per entrenar els models basats en TF-IDF, averaged WV i TFIDF-weigthed WV.

In [109]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

modelLR = LogisticRegression(solver='liblinear')
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
modelRBFSVM = SVC(gamma='scale', C=2)

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM),
           ('Gauss kernel SVM', modelRBFSVM)]

metricas = []
resultados = []

# Models amb característiques BoW.
bow_train_features = bow_train_features.toarray()
bow_test_features = bow_test_features.toarray()

# Models amb característiques averaged word vectors.
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels)
    metrica['modelo'] = f'{m} Averaged Word Vectors'
    resultados.append(prediccion)
    metricas.append(metrica)

# Models amb característiques tf idf weighted average word vectors.
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_wv_test_features,
                                           test_labels=test_labels)
    metrica['modelo'] = f'{m} TF-IDF Weighted WV'
    resultados.append(prediccion)
    metricas.append(metrica)


Converteix la llista de mètriques en un DataFrame per observar-ne els valors:

In [114]:
# COMPLETAR
# Convertir la lista de métricas en un DataFrame
metricas_df = pd.DataFrame(metricas)

# Mostrar el DataFrame
print(metricas_df)


   Accuracy  Precision  Recall  F1 Score  \
0     0.518      0.522   0.518     0.514   
1     0.298      0.324   0.298     0.292   
2     0.468      0.539   0.468     0.452   
3     0.463      0.470   0.463     0.446   
4     0.383      0.392   0.383     0.382   
5     0.265      0.293   0.265     0.236   
6     0.360      0.490   0.360     0.338   
7     0.398      0.405   0.398     0.376   

                                      modelo  
0  Logistic Regression Averaged Word Vectors  
1          Naive Bayes Averaged Word Vectors  
2           Linear SVM Averaged Word Vectors  
3     Gauss kernel SVM Averaged Word Vectors  
4     Logistic Regression TF-IDF Weighted WV  
5             Naive Bayes TF-IDF Weighted WV  
6              Linear SVM TF-IDF Weighted WV  
7        Gauss kernel SVM TF-IDF Weighted WV  


Ordena les mètriques per `accuracy` i mostra el millor resultat:

In [116]:
# COMPLETAR
# Ordenar las métricas por accuracy en orden descendente
metricas_df_sorted = metricas_df.sort_values(by='Accuracy', ascending=False)

# Mostrar el mejor resultado (la primera fila después de ordenar)
mejor_resultado = metricas_df_sorted.iloc[0]

# Imprimir el mejor resultado
print("Mejor resultado:")
print(mejor_resultado)


Mejor resultado:
Accuracy                                         0.518
Precision                                        0.522
Recall                                           0.518
F1 Score                                         0.514
modelo       Logistic Regression Averaged Word Vectors
Name: 0, dtype: object


Visualitza la matriu de confusió de la predicció per al millor model:

In [121]:
# Matriu de confusió
import pandas as pd

indice_mejor_modelo = metricas_df_sorted.index[0]
cm = metrics.confusion_matrix(test_labels, resultados[indice_mejor_modelo])
pd.DataFrame(cm, index=dataset.target_names, columns=modelSVM.classes_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
alt.atheism,9,1,0,0,0,0,1,1,0,0,0,0,0,0,0,5,1,2,3,4
comp.graphics,0,16,5,1,2,5,0,0,0,0,0,0,1,0,0,0,1,0,1,0
comp.os.ms-windows.misc,0,4,14,2,1,4,1,0,0,0,0,0,1,0,0,0,0,0,0,0
comp.sys.ibm.pc.hardware,0,2,4,21,4,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
comp.sys.mac.hardware,0,2,2,7,10,1,1,1,0,0,0,2,5,0,0,0,0,1,0,0
comp.windows.x,0,4,4,1,1,15,0,0,0,0,0,1,1,0,1,0,0,0,0,0
misc.forsale,0,0,0,2,1,0,16,3,2,0,0,0,3,1,0,0,0,2,0,0
rec.autos,0,0,0,1,0,0,0,12,4,0,0,0,2,0,1,0,1,0,0,0
rec.motorcycles,1,1,0,1,0,0,0,1,13,1,0,0,1,0,1,0,0,0,1,0
rec.sport.baseball,0,0,1,0,1,1,0,0,0,18,5,0,1,3,0,0,0,1,1,1


### Exercici.
Divideix el conjunt total de mostres (`corpus` i `labels`) en entrenament i test (30%) i entrena el millor model obtingut, per veure si es milloren els resultats:

In [ ]:
# COMPLETAR